<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/cover-small.jpg">

*Este libro es una versión al español de [Python for Everybody](https://www.py4e.com/) escrito por el [Dr. Charles R. Severance](http://www.dr-chuck.com/); este contenido esta disponible en [GitHub](https://github.com/csev/py4e).*

Detalles de Copyright

*Copyright ~ 2009- Charles Severance.
Este trabajo está registrado bajo una Licencia Creative Commons AttributionNonCommercial-ShareAlike 3.0 [CC BY-NC-SA](https://creativecommons.org/licenses/by-nc-sa/3.0/).*

<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 11 - Expresiones regulares](cap11.ipynb) | [Capítulo 13 - Python y servicios web](cap13.ipynb) >

# Capítulo 12 - Programas en red

Si bien muchos de los ejemplos de este libro se han centrado en leer archivos y buscar datos en esos archivos, hay muchas fuentes diferentes de información cuando uno considera Internet.

En este capítulo pretendemos ser un navegador web y recuperar páginas web utilizando el protocolo de transferencia de hipertexto (HTTP). Luego leeremos los datos de la página web y lo analizaremos.

## Protocolo de transferencia de hipertexto – HTTP

El protocolo de red que alimenta la web es en realidad bastante simple y hay soporte integrado en Python llamado, sockets que hace que sea muy fácil hacer conexiones de red y recuperar datos sobre esos sockets en un programa de Python.

Un socket es muy similar a un archivo, excepto que un solo socket proporciona una conexión bidireccional entre dos programas. Puede leer y escribir en el mismo socket. Si escribe algo en un socket, se envía a la aplicación en el otro extremo del socket. Si lee desde el socket, se le proporcionan los datos que la otra aplicación ha enviado.

Pero si intenta leer un socket cuando el programa en el otro extremo del socket no ha enviado ningún dato, simplemente siéntese y espere. Si los programas en ambos extremos del socket simplemente esperan algunos datos sin enviar nada, esperarán durante mucho tiempo.

Entonces, una parte importante de los programas que se comunican a través de Internet es tener algún tipo de protocolo. Un protocolo es un conjunto de reglas precisas que determinan quién debe ir primero, qué deben hacer y, a continuación, cuáles son las respuestas a ese mensaje y quién envía a continuación, y así sucesivamente. En cierto sentido, las dos aplicaciones en cada extremo del socket están haciendo un baile y asegurándose de no pisar los dedos del otro.

Hay muchos documentos que describen estos protocolos de red. El protocolo de transferencia de hipertexto se describe en el siguiente [documento](https://www.w3.org/Protocols/rfc2616/rfc2616.txt).

Este es un documento largo y complejo de 176 páginas con muchos detalles. Si le parece interesante, siéntase libre de leerlo todo. Pero si echas un vistazo a la página 36 de RFC2616, encontrarás la sintaxis para la solicitud GET. Para solicitar un documento de un servidor web, hacemos una conexión con el servidor `www.pr4e.org` en el puerto 80 y luego enviamos una línea del formulario.

    GET http://data.pr4e.org/romeo.txt HTTP/1.0

donde el segundo parámetro es la página web que estamos solicitando, y luego también enviamos una línea en blanco. El servidor web responderá con cierta información de encabezado sobre el documento y una línea en blanco seguida del contenido del documento.

## El navegador web mas simple del mundo

Quizás la forma más fácil de mostrar cómo funciona el protocolo HTTP es escribir un programa Python muy simple que hace una conexión a un servidor web y sigue las reglas del protocolo HTTP para solicitar un documento y mostrar lo que el servidor envía de vuelta.

In [1]:
import socket

misocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
misocket.connect(('data.pr4e.org', 80))
cmd = 'GET http://data.pr4e.org/romeo.txt HTTP/1.0\r\n\r\n'.encode()
misocket.send(cmd)

while True:
    datos = misocket.recv(512)
    if len(datos) < 1:
        break
    print(datos.decode(),end='')

misocket.close()

HTTP/1.1 200 OK
Date: Sat, 18 Jan 2020 19:22:11 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Sat, 13 May 2017 11:22:22 GMT
ETag: "a7-54f6609245537"
Accept-Ranges: bytes
Content-Length: 167
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: text/plain

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


Primero, el programa establece una conexión con el puerto 80 en el servidor `www.py4e.com` . Como nuestro programa cumple la función de "navegador web", el protocolo HTTP dice que debemos enviar el comando GET seguido de una línea en blanco.

![Figura 12.1](./figures/12.1.svg)
<center><i>Figura 12.1: Una conexión de socket</i></center>

Una vez que enviamos esa línea en blanco, escribimos un bucle que recibe datos en trozos de 512 caracteres del socket e imprime los datos hasta que no hay más datos para leer (es decir, el `recv()` devuelve una cadena vacía).

La salida comienza con los encabezados que envía el servidor web para describir el documento. Por ejemplo, el encabezado `Content-Type` indica que el documento es un documento de texto plano (text/plain).

Después de que el servidor nos envíe los encabezados, agrega una línea en blanco para indicar el final de los encabezados y luego envía los datos reales del archivo `romeo.txt`.

Este ejemplo muestra cómo hacer una conexión de red de bajo nivel con sockets. Los sockets se pueden usar para comunicarse con un servidor web o con un servidor de correo u otros tipos de servidores. Todo lo que se necesita es encontrar el documento que describe el protocolo y escribir el código para enviar y recibir los datos de acuerdo con el protocolo.

Sin embargo, dado que el protocolo que utilizamos más comúnmente es el protocolo web HTTP, Python tiene una biblioteca especial diseñada específicamente para admitir el protocolo HTTP para la recuperación de documentos y datos en la web.

Uno de los requisitos para usar el protocolo HTTP es la necesidad de enviar y recibir datos como objetos de bytes, en lugar de cadenas. En el ejemplo anterior, los métodos `encode()` y `decode()` convierten cadenas en objetos de bytes y viceversa.

El siguiente ejemplo usa notación `b''` para especificar que una variable debe almacenarse como un objeto de bytes. `encode()` y `b''` son equivalentes.

In [2]:
print(b'Hello world')
print('Hello world'.encode())

b'Hello world'
b'Hello world'


## Recuperando una imagen con HTTP

En el ejemplo anterior, recuperamos un archivo de texto sin formato que tenía nuevas líneas en el archivo y simplemente copiamos los datos en la pantalla mientras se ejecutaba el programa. Podemos usar un programa similar para recuperar una imagen usando HTTP. En lugar de copiar los datos en la pantalla mientras se ejecuta el programa, acumulamos los datos en una cadena, recortamos los encabezados y luego guardamos los datos de la imagen en un archivo de la siguiente manera:

In [3]:
import socket
import time

HOST = 'data.pr4e.org'
PUERTO = 80
misocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
misocket.connect((HOST, PUERTO))
misocket.sendall(b'GET http://data.pr4e.org/cover3.jpg HTTP/1.0\r\n\r\n')
count = 0
imagen = b""

while True:
    datos = misocket.recv(5120)
    if len(datos) < 1: break
    #time.sleep(0.25)
    count = count + len(datos)
    print(len(datos), count)
    imagen = imagen + datos

misocket.close()

# Look for the end of the header (2 CRLF)
pos = imagen.find(b"\r\n\r\n")
print('Header length', pos)
print(imagen[:pos].decode())

# Skip past the header and save the picture data
imagen = imagen[pos+4:]
fhand = open("stuff.jpg", "wb")
fhand.write(imagen)
fhand.close()

5120 5120
1980 7100
5120 12220
1980 14200
5120 19320
560 19880
5120 25000
560 25560
5120 30680
4820 35500
2840 38340
2840 41180
5120 46300
1980 48280
4260 52540
5120 57660
3400 61060
5120 66180
3400 69580
1420 71000
5120 76120
1980 78100
2840 80940
2840 83780
1420 85200
2840 88040
4260 92300
5120 97420
1980 99400
5120 104520
4820 109340
1420 110760
5120 115880
560 116440
5120 121560
4820 126380
4260 130640
5120 135760
5120 140880
1120 142000
5120 147120
3400 150520
4260 154780
2840 157620
1420 159040
5120 164160
1800 165960
5120 171080
5120 176200
3960 180160
5120 185280
3400 188680
2840 191520
4440 195960
5120 201080
5120 206200
5120 211320
5120 216440
3660 220100
1420 221520
1420 222940
1420 224360
1420 225780
4260 230040
568 230608
Header length 394
HTTP/1.1 200 OK
Date: Sat, 18 Jan 2020 19:22:11 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Mon, 15 May 2017 12:27:40 GMT
ETag: "38342-54f8f2e5b6277"
Accept-Ranges: bytes
Content-Length: 230210
Vary: Accept-Encoding
Cache-Control: 

Puede ver que para esta url, el encabezado Content-Type indica que el cuerpo del documento es una imagen (image/jpeg). Una vez que se completa el programa, puede ver los datos de la imagen abriendo el archivo stuff.jpg en un visor de imágenes.

A medida que el programa se ejecuta, puede ver que no obtenemos 5120 caracteres cada vez que llamamos al método `recv()`. Obtenemos todos los caracteres que el servidor web nos transfirió a través de la red en el momento en que llamamos `recv()`. En este ejemplo, obtenemos 1460 o 2920 caracteres cada vez que solicitamos hasta 5120 caracteres de datos.

Sus resultados pueden ser diferentes dependiendo de la velocidad de su red. También tenga en cuenta que en la última llamada a `recv()` recibimos 1681 bytes, que es el final de la secuencia, y en la siguiente llamada `recv()` obtenemos una cadena de longitud cero que nos dice que el servidor ha llamado a `close()` en su extremo del socket y allí ya no hay más información próxima.

Podemos ralentizar nuestras llamadas `recv()` sucesivas al quitarle el comentario a la llamada `time.sleep()`. De esta manera, esperamos un cuarto de segundo después de cada llamada para que el servidor pueda "adelantarse" y enviarnos más datos antes de volver a llamar `recv()`. Con el retraso, el programa se ejecuta de la siguiente manera:

In [4]:
import socket
import time

HOST = 'data.pr4e.org'
PUERTO = 80
misocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
misocket.connect((HOST, PUERTO))
misocket.sendall(b'GET http://data.pr4e.org/cover3.jpg HTTP/1.0\r\n\r\n')
count = 0
imagen = b""

while True:
    datos = misocket.recv(5120)
    if len(datos) < 1: break
    time.sleep(0.25)
    count = count + len(datos)
    print(len(datos), count)
    imagen = imagen + datos

misocket.close()

# Look for the end of the header (2 CRLF)
pos = imagen.find(b"\r\n\r\n")
print('Header length', pos)
print(imagen[:pos].decode())

# Skip past the header and save the picture data
imagen = imagen[pos+4:]
fhand = open("stuff.jpg", "wb")
fhand.write(imagen)
fhand.close()

5120 5120
5120 10240
5120 15360
5120 20480
5120 25600
5120 30720
5120 35840
5120 40960
5120 46080
5120 51200
5120 56320
5120 61440
5120 66560
5120 71680
5120 76800
5120 81920
5120 87040
5120 92160
5120 97280
5120 102400
5120 107520
5120 112640
5120 117760
5120 122880
5120 128000
5120 133120
5120 138240
5120 143360
5120 148480
5120 153600
5120 158720
5120 163840
5120 168960
5120 174080
5120 179200
5120 184320
5120 189440
5120 194560
5120 199680
5120 204800
5120 209920
5120 215040
5120 220160
5120 225280
5120 230400
208 230608
Header length 394
HTTP/1.1 200 OK
Date: Sat, 18 Jan 2020 19:22:12 GMT
Server: Apache/2.4.18 (Ubuntu)
Last-Modified: Mon, 15 May 2017 12:27:40 GMT
ETag: "38342-54f8f2e5b6277"
Accept-Ranges: bytes
Content-Length: 230210
Vary: Accept-Encoding
Cache-Control: max-age=0, no-cache, no-store, must-revalidate
Pragma: no-cache
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Connection: close
Content-Type: image/jpeg


Ahora, aparte de la última llamada a `recv()`, ahora obtenemos 5120 caracteres cada vez que solicitamos nuevos datos.

Hay un búfer entre las solicitudes `send()` de realización del servidor y nuestras solicitudes de realización de aplicaciones `recv()`. Cuando ejecutamos el programa con la demora en su lugar, en algún momento el servidor puede llenar el búfer en el socket y verse obligado a pausar hasta que nuestro programa comience a vaciar el búfer. La pausa de la aplicación de envío o la aplicación de recepción se denomina "control de flujo".

## Recuperando páginas web con `urllib`

Si bien podemos enviar y recibir datos manualmente a través de HTTP utilizando la biblioteca de socket, hay una manera mucho más simple de realizar esta tarea común en Python al usar la biblioteca `urllib`.

Usando `urllib`, puedes tratar una página web como un archivo. Simplemente indica qué página web desea recuperar y `urllib` maneja todos los detalles del encabezado y el protocolo HTTP.

El código equivalente para leer el archivo `romeo.txt` desde la web con `urllib` es el siguiente:

In [5]:
import urllib.request

fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')
for linea in fhand:
    print(linea.decode().strip())

But soft what light through yonder window breaks
It is the east and Juliet is the sun
Arise fair sun and kill the envious moon
Who is already sick and pale with grief


Una vez que se ha abierto la página web usando `urllib.urlopen`, podemos tratarla como un archivo y leerla mediante un bucle `for`.

Cuando se ejecuta el programa, solo vemos el resultado del contenido del archivo. Los encabezados aún se envían, pero el código `urllib` suprime los encabezados y solo nos devuelve los datos.

Como ejemplo, podemos escribir un programa para recuperar los datos `romeo.txt` y calcular la frecuencia de cada palabra en el archivo de la siguiente manera:

In [6]:
import urllib.request, urllib.parse, urllib.error

fhand = urllib.request.urlopen('http://data.pr4e.org/romeo.txt')
counts = dict()
for linea in fhand:
    palabras = linea.decode().split()
    for palabra in palabras:
        counts[palabra] = counts.get(palabra, 0) + 1
print(counts)

{'But': 1, 'soft': 1, 'what': 1, 'light': 1, 'through': 1, 'yonder': 1, 'window': 1, 'breaks': 1, 'It': 1, 'is': 3, 'the': 3, 'east': 1, 'and': 3, 'Juliet': 1, 'sun': 2, 'Arise': 1, 'fair': 1, 'kill': 1, 'envious': 1, 'moon': 1, 'Who': 1, 'already': 1, 'sick': 1, 'pale': 1, 'with': 1, 'grief': 1}


Nuevamente, una vez que hemos abierto la página web, podemos leerla como un archivo local.

## Lectura de archivos binarios usando `urllib`

En ocasiones, desea recuperar un archivo que no sea de texto (o binario), como un archivo de imagen o video. Los datos en estos archivos generalmente no son útiles para imprimir, pero puede hacer fácilmente una copia de una URL a un archivo local en su disco duro usando `urllib`.

El patrón es abrir la URL y usarla `read` para descargar todo el contenido del documento en una variable de cadena (img) y luego escribir esa información en un archivo local de la siguiente manera:

In [7]:
import urllib.request, urllib.parse, urllib.error

img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg').read()
fhand = open('cover3.jpg', 'wb')
fhand.write(img)
fhand.close()

Este programa lee todos los datos a la vez a través de la red y los almacena en la variable `img` en la memoria principal de su computadora, luego abre el archivo `cover3.jpg` y escribe los datos en su disco. El argumento `wb` para open() abre un archivo binario para escribir solo. Este programa funcionará si el tamaño del archivo es menor que el tamaño de la memoria de su computadora.

Sin embargo, si se trata de un archivo de audio o video de gran tamaño, este programa puede bloquearse o al menos ejecutarse extremadamente lento cuando la computadora se queda sin memoria. Para evitar quedarse sin memoria, recuperamos los datos en bloques (o buffers) y luego escribimos cada bloque en su disco antes de recuperar el siguiente bloque. De esta manera, el programa puede leer archivos de cualquier tamaño sin utilizar toda la memoria que tiene en su computadora.

In [8]:
import urllib.request, urllib.parse, urllib.error

img = urllib.request.urlopen('http://data.pr4e.org/cover3.jpg')
fhand = open('cover3.jpg', 'wb')
size = 0
while True:
    info = img.read(100000)
    if len(info) < 1: break
    size = size + len(info)
    fhand.write(info)

print(size, 'carácteres coiados.')
fhand.close()

230210 carácteres coiados.


En este ejemplo, leemos solo 100,000 caracteres a la vez y luego los escribimos en el archivo `cover.jpg` antes de recuperar los siguientes 100,000 caracteres de datos de la web.

## Analizando HTML y raspando la web

Uno de los usos comunes de `urllib` en Python es la capacidad de "raspar la web" también conocido como "web scraping". El raspado web es cuando escribimos un programa que pretende ser un buscador web y recupera páginas, luego examina los datos en esas páginas
buscando patrones.

Como ejemplo, un motor de búsqueda como Google buscará en el origen de una página web y extraerá los enlaces a otras páginas y recuperará esas páginas, extraerá enlaces, y así sucesivamente. Con esta técnica, Google navega a través de casi todas las páginas de la web.

Google también usa la frecuencia de los enlaces de las páginas que encuentra en una página en particular como una medida de cuán "importante" es una página y qué tan alta debe aparecer la página en sus resultados de búsqueda.

## Análisis de HTML utilizando expresiones regulares

Una forma simple de analizar HTML es usar expresiones regulares para buscar y extraer subcadenas que coincidan con un patrón en particular. Aquí hay una página web simple:

    <h1>The First Page</h1>
    <p>
    If you like, you can switch to the
    <a href="http://www.dr-chuck.com/page2.htm">
    Second Page</a>.
    </p>

Podemos construir una expresión regular bien formada para unir y extraer los valores del enlace del texto anterior de la siguiente manera:

    href="http://.+?"
    
Nuestra expresión regular busca cadenas que comiencen con `href="http://`, seguido de uno o más caracteres `.+?`, Seguido de otra comilla doble. El signo de interrogación agregado a `". +? "` indica que el partido se debe hacer de una manera "no codiciosa" en lugar de "codiciosa". Un partido no codicioso intenta encontrar la secuencia de combinación más pequeña posible y un partido codicioso intenta encontrar la secuencia de mayor coincidencia posible.

Agregamos paréntesis a nuestra expresión regular para indicar qué parte de nuestra cadena coincidente nos gustaría extraer y generar el siguiente programa:

In [9]:
# Busqueda de lineas que inicien con From y tengan un @
import urllib.request, urllib.parse, urllib.error
import re

url = input('Enter - ')
html = urllib.request.urlopen(url).read()
links = re.findall(b'href="(http://.*?)"', html)
for link in links:
    print(link.decode())

Enter - https://docs.python.org
http://sphinx.pocoo.org/


Las expresiones regulares funcionan muy bien cuando su HTML está bien formateado y es predecible. Pero dado que hay muchas páginas HTML "rotas", una solución que solo utiliza expresiones regulares puede omitir algunos enlaces válidos o terminar con datos incorrectos.

Esto se puede resolver utilizando una robusta biblioteca de análisis HTML.

## Análisis de HTML usando BeautifulSoup

Hay varias bibliotecas de Python que pueden ayudarlo a analizar HTML y extraer datos de las páginas. Cada una de las bibliotecas tiene sus fortalezas y debilidades y puede elegir una según sus necesidades.

Como ejemplo, simplemente analizaremos algunos enlaces de entrada y extracción de HTML utilizando la
biblioteca `BeautifulSoup`. Puede descargar e instalar el código BeautifulSoup desde: http://www.crummy.com/software/.

Puede descargar e "instalar" `BeautifulSoup` o simplemente puede colocar el archivo BeautifulSoup.py en la misma carpeta que su aplicación.

Aunque el HTML se parece a XML y algunas páginas están cuidadosamente construidas para ser XML, la mayoría de los HTML se suelen romper de tal forma que hacen que un analizador XML rechace la página completa de HTML como inadecuada. `BeautifulSoup` tolera HTML altamente defectuoso y aún le permite extraer fácilmente los datos que necesita.

Usaremos `urllib` para leer la página y luego usaremos `BeautifulSoup` para extraer los atributos `href` de las etiquetas de anclaje.

In [10]:
# Para ejecutar esto necesitas intallar BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# o descargar el archivo
# http://www.py4e.com/code3/bs4.zip
# y descomprimirlo en el mismo directorio de este archivo

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignora errores de certificados SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Ingresa un url - ')
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all of the anchor tags
tags = soup('a')
for tag in tags:
    print(tag.get('href', None))

Ingresa un url - https://docs.python.org
genindex.html
py-modindex.html
https://www.python.org/
#
whatsnew/3.8.html
whatsnew/index.html
tutorial/index.html
library/index.html
reference/index.html
using/index.html
howto/index.html
installing/index.html
distributing/index.html
extending/index.html
c-api/index.html
faq/index.html
py-modindex.html
genindex.html
glossary.html
search.html
contents.html
bugs.html
https://devguide.python.org/docquality/#helping-with-documentation
about.html
license.html
copyright.html
download.html
https://docs.python.org/3.9/
https://docs.python.org/3.8/
https://docs.python.org/3.7/
https://docs.python.org/3.6/
https://docs.python.org/3.5/
https://docs.python.org/2.7/
https://www.python.org/doc/versions/
https://www.python.org/dev/peps/
https://wiki.python.org/moin/BeginnersGuide
https://wiki.python.org/moin/PythonBooks
https://www.python.org/doc/av/
https://devguide.python.org/
genindex.html
py-modindex.html
https://www.python.org/
#
copyright.html
https://w

El programa solicita una dirección web, luego abre la página web, lee los datos y los pasa al analizador `BeautifulSoup`, y luego recupera todas las etiquetas de anclaje e imprime el atributo `href` para cada etiqueta.

Esta lista es mucho más larga porque algunas etiquetas de anclaje HTML son rutas relativas (por ejemplo, tutorial/index.html) o referencias en la página (por ejemplo, '#') que no incluyen "http://" o "https://", que era un requisito en nuestra expresión regular.

También puede usar `BeautifulSoup` para extraer varias partes de cada etiqueta:

In [11]:
# Para ejecutar esto necesitas intallar BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# o descargar el archivo
# http://www.py4e.com/code3/bs4.zip
# y descomprimirlo en el mismo directorio de este archivo

from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl

# Ignora errores de certificados SSL
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = input('Ingresa un url - ')
html = urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, "html.parser")

# Recupera todas las etiquetas de anclaje
tags = soup('a')
for tag in tags:
    # Look at the parts of a tag
    print('TAG:', tag)
    print('URL:', tag.get('href', None))
    print('Contents:', tag.contents[0])
    print('Attrs:', tag.attrs)

Ingresa un url - http://www.dr-chuck.com/page1.htm
TAG: <a href="http://www.dr-chuck.com/page2.htm">
Second Page</a>
URL: http://www.dr-chuck.com/page2.htm
Contents: 
Second Page
Attrs: {'href': 'http://www.dr-chuck.com/page2.htm'}


`html.parser` es el analizador HTML incluido en la biblioteca estándar de Python 3. La información sobre otros analizadores HTML está disponible en: http://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser. Estos ejemplos solo comienzan a mostrar el poder de `BeautifulSoup` cuando se trata de analizar HTML.

## Comandos `curl`

Si tiene una computadora Linux, Unix, Macintosh, o Windows 10 es probable que tenga comandos integrados en su sistema operativo que recupere tanto texto sin formato como archivos binarios utilizando los protocolos HTTP o File Transfer (FTP). Uno de estos comandos es `curl`:

    $ curl -O http://www.py4e.com/cover.jpg

El comando `curl` es la abreviatura de "copiar URL" y, por lo tanto, los dos ejemplos enumerados anteriormente para recuperar archivos binarios con urllib tienen un nombre inteligente `curl1.py` y `curl2.py`, ya que implementan una funcionalidad similar al comando `curl`. También hay un programa de muestra llamado `curl3.py` que realiza esta tarea de manera un poco más efectiva, en caso de que realmente quiera usar este patrón en un programa que está escribiendo.

Un segundo comando que funciona de manera muy similar es `wget` (no esta disponible en Windows):

    $ wget http://www.py4e.com/cover.jpg
    
Ambos comandos hacen que la recuperación de páginas web y archivos remotos sea una tarea simple.

## Glosario

* **BeautifulSoup:** Una biblioteca de Python para analizar documentos HTML y extraer datos de documentos HTML que compensa la mayoría de las imperfecciones en el HTML que los navegadores generalmente ignoran. Puede descargar el código de BeautifulSoup desde www.crummy.com .
* **puerto:** Un número que generalmente indica con qué aplicación se está contactando cuando realiza una conexión de socket a un servidor. Como ejemplo, el tráfico web generalmente usa el puerto 80 mientras que el tráfico de correo electrónico usa el puerto 25.
* **raspar:** Cuando un programa pretende ser un navegador web y recupera una página web, luego mira el contenido de la página web. A menudo, los programas siguen los enlaces en una página para encontrar la página siguiente y poder atravesar una red de páginas o una red social.
* **socket:** Una conexión de red entre dos aplicaciones donde las aplicaciones pueden enviar y recibir datos en cualquier dirección.
* **araña:** El acto de un motor de búsqueda web recuperando una página y luego todas las páginas enlazadas desde una página y así sucesivamente hasta que tengan casi todas las páginas en Internet que usan para construir su índice de búsqueda.

## Ejercicios

**Ejercicio 1:** cambie el programa de socket `socket1.py` para solicitar al usuario la URL para que pueda leer cualquier página web. Puede usar `split('/')` para dividir la URL en partes para poder extraer el nombre de host para la llamada del socket `connect`. Agregue la comprobación de errores usando `try` y `except` para manejar la condición en la que el usuario ingresa una URL con formato incorrecto o inexistente.

**Ejercicio 2:** cambie su programa de socket para que cuente el número de caracteres que ha recibido y deje de mostrar cualquier texto después de que haya mostrado 3000 caracteres. El programa debe recuperar el documento completo y contar el número total de caracteres y mostrar el recuento del número de caracteres al final del documento.

**Ejercicio 3:** se utiliza `urllib` para replicar el ejercicio anterior de (1) recuperar el documento de una URL, (2) mostrar hasta 3000 caracteres y (3) contar el número total de caracteres en el documento. No se preocupe por los encabezados de este ejercicio, simplemente muestre los primeros 3000 caracteres del contenido del documento.

**Ejercicio 4:** cambie el programa `urllinks.py` para extraer y contar etiquetas de párrafo (p) del documento HTML recuperado y mostrar el recuento de los párrafos como la salida de su programa. No debes mostrar el texto del párrafo, solo contarlos. Pruebe su programa en varias páginas web pequeñas, así como en algunas páginas web más grandes.

**Ejercicio 5:** (Avanzado) Cambie el programa de socket para que solo muestre datos después de que se hayan recibido los encabezados y una línea en blanco. Recuerde que `recv` recibe caracteres (líneas nuevas y todo), no líneas.



<!--NAVIGATION-->
| [Indice](indice.ipynb) | 

< [Capítulo 11 - Expresiones regulares](cap11.ipynb) | [Capítulo 13 - Python y servicios web](cap13.ipynb) >